In [65]:
"""
@authors: faurand, chardes, ehagensieker
"""
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import math
import datetime
import tqdm
import pprint

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

#load the mnist dataset
(train_ds, test_ds) = tfds.load('Cifar10', split=['train', 'test'], as_supervised=True)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [66]:
def prepare_data(batch_size, data): 
  '''
  prepare the dataset to have one-hot-vectors and values between -1 and 1
  '''
  data = data.map(lambda img, target: (tf.cast(img, tf.float32), tf.cast(target,tf.int32)))
  data = data.map(lambda img, target: ((img/128.)-1.0, target))
  data = data.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  
  data = data.cache()
  data = data.shuffle(1000)
  data = data.batch(batch_size)
  data = data.prefetch(20)

  return data


In [67]:
from tensorflow.keras.layers import Dense

class BasicConv(tf.keras.Model):
  '''
  create a small network with 2 blocks each having 2 layers and starting with 32 filters in the first layer
  '''
  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConv, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [68]:
from tensorflow.keras.layers import Dense

class BasicConvFilter1(tf.keras.Model):
  '''
  create a small network with 2 blocks each having 2 layers and starting with 64 filters in the first layer
  '''
  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConvFilter1, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [69]:
from tensorflow.keras.layers import Dense

class BasicConvFilter2(tf.keras.Model):
  '''
  create a small network with 2 blocks each having 2 layers and starting with 16 filters in the first layer
  '''
  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConvFilter2, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [70]:
from tensorflow.keras.layers import Dense

class BasicConvDeep(tf.keras.Model):
  '''
  create a deeper network with 4 blocks each having 2 layers and starting with 32 filters in the first layer
  '''

  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConvDeep, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.l5 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l6 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.l7 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l8 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.pooling1(x)
    x = self.l5(x)
    x = self.l6(x)
    x = self.pooling2(x)
    x = self.l7(x)
    x = self.l8(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [71]:
class DenselyConnectedCNNLayer(tf.keras.layers.Layer):
  def __init__(self, num_filter): 
    super(DenselyConnectedCNNLayer, self).__init__()

    self.conv = tf.keras.layers.Conv2D(filters = num_filter, kernel_size = 3, padding = "same", activation = "relu")

  def call(self,x): 
    c = self.conv(x)
    #concatenate at last layer - get long stack of all earlier layer
    x = tf.concat((x,c), axis = -1)
    return x


class DenselyConnectedCNNBlock(tf.keras.layers.Layer):
  #take all conv layers and create a block we take for one convolution
  def __init__(self, num_filters, layers):
    super(DenselyConnectedCNNBlock, self).__init__()
    self.layers = [DenselyConnectedCNNLayer(num_filters) for _ in range(layers)]

  def call(self, x): 
    #take input and pump it through all layers we have 
    for l in self.layers:
      x = l(x)
      return x


class DenselyConnectedCNN(tf.keras.Model):
  def __init__(self, optimizer = tf.keras.optimizers.Adam()):
    super(DenselyConnectedCNN, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #4 layers with each of 24 kernels -> get 4 layer in 28*28 resolution 
    self.denseblock1 = DenselyConnectedCNNBlock(24,4)
    #go down to 14*14
    self.pooling1 = tf.keras.layers.MaxPooling2D()

    self.denseblock2 = DenselyConnectedCNNBlock(24,4)
    self.pooling2 = tf.keras.layers.MaxPooling2D()

    self.denseblock3 = DenselyConnectedCNNBlock(24,4)
    self.globalpooling = tf.keras.layers.GlobalAvgPool2D()
    self.out = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.denseblock1(x)
    x = self.pooling1(x)
    x = self.denseblock2(x)
    x = self.pooling2(x)
    x = self.denseblock3(x)
    x = self.globalpooling(x)
    x = self.out(x)
    return x

  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}


dense_model = DenselyConnectedCNN()


In [72]:
class ResidualConnectedCNNLayer(tf.keras.layers.Layer):
  def __init__(self, num_filters, optimizer = tf.keras.optimizers.Adam()):
    super(ResidualConnectedCNNLayer, self).__init__()
    self.conv = tf.keras.layers.Conv2D(filters=num_filters, kernel_size=3, padding='same', activation='relu')

  def call(self, x):
    c = self.conv(x)
    #residual connection 
    x = c+x
    return x

class ResidualConnectedCNNBlock(tf.keras.layers.Layer):
  def __init__(self, depth, layers):
    super(ResidualConnectedCNNBlock, self).__init__()
    #depth increases in the end of the network (cannot add them with different sizes)
    #specify the depth such that they get the same depth 
    self.deeper_layer = tf.keras.layers.Conv2D(filters=depth, kernel_size=3, padding='same', activation='relu')
    #list with many residual connected cnn layer
    self.layers = [ResidualConnectedCNNLayer(depth) for _ in range(layers)]

  def call(self, x):
    #bring it to the correct depth 
    x = self.deeper_layer(x)
    for layer in self.layers:
      x = layer(x)
    return x


class ResidualConnectedCNN(tf.keras.Model):
  def __init__(self, optimizer = tf.keras.optimizers.Adam()):
    super(ResidualConnectedCNN, self).__init__()
    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]
    self.residualblock1 = ResidualConnectedCNNBlock(24,4)
    self.pooling1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.residualblock2 = ResidualConnectedCNNBlock(48,4)
    self.pooling2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.residualblock3 = ResidualConnectedCNNBlock(96,4)
    self.globalpooling = tf.keras.layers.GlobalAvgPool2D()
    
    self.out = tf.keras.layers.Dense(10, activation='softmax')

  def call(self,x):
    x = self.residualblock1(x)
    x = self.pooling1(x)
    x = self.residualblock2(x)
    x = self.pooling2(x)
    x = self.residualblock3(x)
    x = self.globalpooling(x)
    x = self.out(x)
    return x

  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}


resnet = ResidualConnectedCNN()

In [73]:
def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer): 
  '''
  create the training loop where the model is trained on the train dataset and afterwards tested on the validation dataset
  '''
  #iterate over given amount of epochs
  for epoch in range(epochs): 
    print(f"Epoch {epoch}: ")

    #train on all batches of the training data
    for data in tqdm.tqdm(train_ds, position = 0, leave = True):

      metrics = model.train_step(data)


      with train_summary_writer.as_default(): 
        for metric in model.metrics: 
          tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

    # print the metrics
    print([f"Train {key}: {value.numpy()}" for (key, value) in metrics.items()])

    #reset metric 
    model.reset_metrics()
  
    #evaluation on validation set
    for data in val_ds:
      metrics = model.test_step(data)

      with val_summary_writer.as_default():
        for metric in model.metrics:
          tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
      
    # print the metrics
    print([f"Test {key}: {value.numpy()}" for (key, value) in metrics.items()])

    #reset metric
    model.reset_metric()

    print("\n")

In [74]:
def create_summary_writers(config_name):
  '''
  create the summary writer to have access to the metrics of the model 
  '''
  current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  train_log_path = f"logs/submission/{config_name}/{current_time}/train"
  val_log_path = f"logs/submission/{config_name}/{current_time}/val"

  # log writer
  train_summary_writer = tf.summary.create_file_writer(train_log_path)
  val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
  return train_summary_writer, val_summary_writer

In [80]:
import matplotlib.pyplot as plt 
def run(model):
  '''
  run the model and specify the number of epochs, batch_size and prepare the training and testing data
  '''

  ### Hyperparameters
  num_epochs = 10
  batch_size = 32
  train = prepare_data(batch_size,train_ds)
  test = prepare_data(batch_size,test_ds)

  train_summary_writer, val_summary_writer = create_summary_writers(config_name = f'RUN')
  training_loop(model,train,test,num_epochs,train_summary_writer, val_summary_writer)
  # print(train_summary_writer)

  # #create two different plots for losses and accuracies
  # fig, ax = plt.subplots(1, 2)

  # #plot the losses
  # ax[0].plot(train_summary_writer.scalar(), "b-", label = "train loss")
  # ax[0].plot(val_summary_writer.scalar(), "r-", label = "test_loss")
  # ax[0].legend(loc = "best")
  # ax[0].set_xlabel("Training steps")
  # ax[0].set_ylabel("Loss performance")

  
  # #move the plot to the right (to not have the label inside the ax[0] plot)
  # # box = ax[1].get_position() 
  # # box.x0 = box.x0 + 0.05
  # # box.x1 = box.x1 + 0.05
  # # ax[1].set_position(box)

  # #set the title 
  # fig.suptitle("Performance of the model")

  # plt.show()


In [ ]:
myMod1 = BasicConv(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))

myMod2 = BasicConv(tf.keras.optimizers.Adam(learning_rate=0.001))
myMod3 = BasicConv(tf.keras.optimizers.RMSprop(learning_rate=0.001))
myMod4 = BasicConv(tf.keras.optimizers.Adagrad(learning_rate=0.001))
myMod5 = BasicConv(tf.keras.optimizers.SGD(learning_rate=0.001))

print("BasicConv, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod1)
print("\nBasicConv, optimizer=Adam, learning_rate=0.001\n")
run(myMod2)
print("BasicConv, optimizer=RMSProp, learning_rate=0.001\n")
run(myMod3)
print("\nBasicConv, optimizer=Adagrad, learning_rate=0.001\n")
run(myMod4)
print("\BasicConv, optimizer=SGD, learning_rate=0.001\n")
run(myMod5)





BasicConv, optimizer=SGD, learning_rate=0.001, momentum=0.9

Epoch 0: 


100%|██████████| 1563/1563 [03:21<00:00,  7.74it/s]


['Train loss: 2.142871379852295', 'Train acc: 0.191880002617836']
['Test loss: 2.0004630088806152', 'Test acc: 0.2551000118255615']


Epoch 1: 


100%|██████████| 1563/1563 [02:50<00:00,  9.15it/s]


['Train loss: 1.9060688018798828', 'Train acc: 0.2815200090408325']
['Test loss: 1.793137788772583', 'Test acc: 0.33500000834465027']


Epoch 2: 


100%|██████████| 1563/1563 [02:51<00:00,  9.12it/s]


['Train loss: 1.7497583627700806', 'Train acc: 0.3366200029850006']
['Test loss: 1.698600172996521', 'Test acc: 0.36579999327659607']


Epoch 3: 


 56%|█████▌    | 877/1563 [01:36<01:14,  9.18it/s]

In [ ]:
myMod6 = BasicConvFilter1(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
myMod7 = BasicConvFilter1(tf.keras.optimizers.Adam(learning_rate=0.001))

print("BasicConvFilter1 with a higher number of filters, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod6)
print("\nBasicConvFilter1 with a higher number of filters, optimizer=Adam, learning_rate=0.001\n")
run(myMod7)

In [ ]:
myMod8 = BasicConvFilter2(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
myMod9 = BasicConvFilter2(tf.keras.optimizers.Adam(learning_rate=0.001))

print("BasicConvFilter2 with a lower number of filters, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod8)
print("\nBasicConvFilter2 with a lower number of filters, optimizer=Adam, learning_rate=0.001\n")
run(myMod9)

In [ ]:
myMod10 = BasicConvDeep(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
myMod11 = BasicConvDeep(tf.keras.optimizers.Adam(learning_rate=0.001))

print("BasicConvDeep with a deeper network, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod10)
print("\nBasicConvDeep with a deeper network, optimizer=Adam, learning_rate=0.001\n")
run(myMod11)

In [ ]:
myMod12 = ResidualConnectedCNN(tf.keras.optimizers.SGD(learning_rate=0.001))

print("Residual Convolutional Network, optimizer=SGD, learning_rate=0.001")
run(myMod12)

In [ ]:
myMod13 = DenselyConnectedCNN(tf.keras.optimizers.SGD(learning_rate=0.001))

print("Densely Connected Convolutional Network, optimizer=SGD\n")
x = run(myMod13)
print(x)


In [ ]:
%tensorboard --logdir logs/submission/